In [2]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [13]:
df = pd.read_parquet('./data/green_tripdata_2022-01.parquet')

df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [14]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

7.652716560267797

 ### mlflow ui --backend-store-uri sqlite:///mlflow.db

In [15]:
import mlflow

mlflow.get_registry_uri()

'file:///workspaces/mlops-zoomcamp-2025/mlruns'

In [16]:
mlflow.set_tracking_uri('sqlite:///mlflow.db')  # Set the tracking URI to a valid SQLite database
mlflow.set_experiment('nyc-taxi-experiment')   # Set the experiment name

<Experiment: artifact_location='/workspaces/mlops-zoomcamp-2025/mlruns/1', creation_time=1745700092379, experiment_id='1', last_update_time=1745700092379, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

## Method 1: Create Pickle file and save it (Not Ideal to load and make inference)

In [18]:
with open('model.bin', 'wb') as f_out:
    pickle.dump((lr,dv), f_out)

In [ ]:
with mlflow.start_run():
    mlflow.log_artifact('model.bin',artifact_path='nyc-trip-model')
    

# Method 2 -Recommended

In [20]:
with mlflow.start_run():
    mlflow.sklearn.log_model(lr, artifact_path='nyc-trip-model',mod)

2025/04/27 22:10:17 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp82zl6z6p/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/04/27 22:10:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


# Load Logged model

In [23]:
import mlflow
logged_model = 'runs:/1e1c6614fe1045d9aa2e740a54a07867/nyc-trip-model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
loaded_model.predict(X_train)

array([ 8.85651993,  8.10497089, 18.64034874, ..., 18.12956425,
       10.11127282, 14.57479921])